In [ ]:
from regelum.data_buffers.data_buffer import DataBuffer
from regelum.data_buffers.batch_sampler import RollingBatchSampler
from regelum.system import InvertedPendulumPD
import numpy as np
import torch
import casadi as cs
from regelum.model import ModelQuadLin, Model
from regelum.__utilities import rc
from typing import Union
import regelum as rg
from regelum.system import ThreeWheeledRobotNI
import pprint
from regelum.predictor import EulerPredictor

In [5]:
import casadi as cs
import numpy as np
import torch

backend = cs.DM


(backend([[1, 1], [2, 2]]).T - backend([[2], [4]])).T ** 2

DM(
[[1, 9], 
 [0, 4]])

In [1]:
import casadi as cs
import numpy as np
import torch

from regelum.constraint import EllipseConstraint, LinearConstraint

In [2]:
constraint1 = EllipseConstraint(coefs=[1, 1, 0], centers=[1, 1, 0], radius=1)
constraint2 = EllipseConstraint(coefs=[2, 2, 0], centers=[0, 0, 0], radius=3)

constraint = constraint1 + constraint2

In [12]:
constraint = LinearConstraint([0, 1, 0], bias=-1) * LinearConstraint(
    [0, -1, 0], bias=-1
)

constraint(cs.DM([[0, -0.5, 0]]))

DM(-0.5)

In [3]:
constraint(cs.MX.sym("x", 2, 3))

MX(min(vertcat(max((ones(2x1)-mac(sq((x'-repmat([1, 1, 0], 2))'),[1, 1, 0],zeros(2x1)))), max((all_9(2x1)-mac(sq((x'-repmat(zeros(3x1), 2))'),[2, 2, 0],zeros(2x1)))))))

In [ ]:
np.array([])

In [ ]:
data_buffer = DataBuffer(20)
pp = pprint.PrettyPrinter(depth=3)

for batch in data_buffer.iter_batches(
    batch_size=4,
    n_batches=2,
    keys=["observation", "action"],
    mode="backward",
    dtype=cs.DM,
):
    print("\nPrinting batch")
    print(
        "Action shape:",
        batch["action"].shape,
        "| Observation shape: ",
        batch["observation"].shape,
    )
    pp.pprint(batch)

In [ ]:
class TestOpt(rg.Optimizable):
    def __init__(self, opt_config):
        super().__init__(opt_config)
        self.create_variable("")

In [23]:
pr = EulerPredictor(ThreeWheeledRobotNI(), 0.01, 1)
x1 = rc.DM([[5.0, 5.0, 2.36]])
acts = rc.DM([[10.0, 0.0]])

x2 = pr.predict_sequence(x1, acts)
x2

DM([[4.92902, 5.07044, 2.36]])

In [ ]:
ModelQuadLin(quad_matrix_type="diaginal", weights=np.array([1, 10, 1, 0, 0]))

In [1]:
from regelum import FunctionWithSignature, VarContainer, Optimizable
from regelum.optimizable.core.configs import casadi_default_config

In [14]:
def f(x, y):
    return x + y

In [31]:
f_signed = FunctionWithSignature(f)

In [20]:
opt = Optimizable(casadi_default_config)
x_var = opt.create_variable(1, 1, name="x")

In [27]:
x_var.as_constant()

OptimizationVariable(name='x', dims=(1, 1), data=None, metadata=MX(opti4_x_1), is_constant=True, hooks=<regelum.optimizable.core.entities.ChainedHook object at 0x7f6e1d350e50>)

In [28]:
x_var.is_constant

True

In [32]:
f_signed.declare_variables(x_var)

FunctionWithSignature(func=<function f at 0x7f6e1d2fdee0>, variables=VarContainer:
  x
  data: None
  metadata: opti4_x_1
  dims: (1, 1)
  is_constant: True


  , is_objective=False, metadata=None)

In [34]:
f_signed.constants_to_substitute

['x']

In [36]:
f_signed.set_parameters(x=1)

In [38]:
f_signed(y=2)

3

In [ ]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Any
from typing import List, Union
from typing_extensions import Self
from regelum import RegelumBase
from regelum.__utilities import rc


class Constraint(RegelumBase, ABC):
    @abstractmethod
    def __call__(self, observation) -> Any:
        pass

    def __add__(self, right: Union[Self, ComposedConstraint]):
        return ComposedConstraint(constraints=[self, right])

    def __mul__(self, right: Union[Self, ComposedConstraint]):
        return ComposedConstraint(constraints=[self, right], is_union=False)


class LinearConstraint(Constraint):
    def __init__(self, weights, bias) -> None:
        self.weights = weights
        self.bias = bias

    def __call__(self, state) -> Any:
        weights_col = rc.force_column(rc.array(self.weights, prototype=state))
        return rc.max(state @ weights_col + self.bias)


class EllipseConstraint(Constraint):
    def __init__(self, coefs, centers, radius) -> None:
        self.coefs = coefs
        self.centers = centers
        self.radius = radius

    def __call__(self, state, coefs, centers, radius) -> Any:
        centers_column = rc.force_column(rc.array(self.centers, prototype=state))
        coefs_column = rc.force_column(rc.array(self.coefs, prototype=state))
        return rc.max(
            self.radius**2 - (state.T - centers_column).T ** 2 @ coefs_column
        )


class ComposedConstraint(Constraint):
    def __init__(self, constraints: List[Constraint], is_union=True):
        self.constraints = constraints
        self.is_union = is_union

    def __call__(self, state) -> Any:
        violations = rc.vstack([constraint(state) for constraint in self.constraints])
        return rc.min(violations) if self.is_union else rc.max(violations)

    def __add__(self, right: Union[Constraint, Self]):
        return ComposedConstraint(constraints=[self, right])

    def __mul__(self, right: Union[Constraint, Self]):
        return ComposedConstraint(constraints=[self, right], is_union=False)

In [1]:
from regelum.optimizable.core.configs import OptimizerConfig
from regelum import FunctionWithSignature, VarContainer, Optimizable
from regelum.optimizable.core.configs import casadi_default_config


class B:
    def __init__(self, var_container, vars_to_handle):
        self.var_container = var_container
        self.vars_to_handle = vars_to_handle

    def push_data_to_vars(self):
        vars_selected = self.var_container.selected(self.vars_to_handle)
        vars_selected.substitute_data(**{name: 1 for name in vars_selected.names})


class A(Optimizable):
    def __init__(self, optimizer_config: OptimizerConfig) -> None:
        super().__init__(optimizer_config)
        self.x = self.create_variable(1, 1, name="x")
        self.y = self.create_variable(1, 1, name="y", is_constant=True)
        self.z = self.create_variable(1, 1, name="z", is_constant=True)
        self.register_objective(self.f, variables=[self.x, self.y, self.z])
        self.register_constraint(self.constr, variables=[self.x, self.y])

    def get_b_instance(self):
        return B(self.variables, ["y", "z"])

    def constr(self, x, y):
        return 2 * y - x

    def f(self, x, y, z):
        return x**2 + y**2 + z**2

In [2]:
a = A(casadi_default_config)

In [3]:
b = a.get_b_instance()

In [4]:
a.y.metadata

MX(opti0_p_1)

In [5]:
b.push_data_to_vars()

In [6]:
# a.optimize(x=10, raw=True)
a.optimize(x=10, raw=True)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 173.00us ( 28.83us)  14.46us (  2.41us)         6
       nlp_g  | 214.00us ( 35.67us)  17.65us (  2.94us)         6
  nlp_grad_f  | 281.00us ( 40.14us)  22.32us (  3.19us)         7
  nlp_hess_l  | 127.00us ( 25.40us)  10.86us (  2.17us)         5
   nlp_jac_g  | 174.00us ( 24.86us)  14.36us (  2.05us)         7
       total  |  85.01ms ( 85.01ms)   7.11ms (  7.11ms)         1


{'constr': DM(-6.26583e-10), 'f': DM(6), 'x': DM(2)}

In [7]:
a.opt_status

'success'

In [8]:
a.constr(3.97054e-05, 1)

1.9999602946

In [18]:
import numpy as np

np.float32(0.2324554634) * np.float32(0.4325234523)

0.10054244

In [9]:
32324554634 * 34325234523

1109547918663576429582

In [9]:
a.constraints

FuncContainer:
  FunctionWithSignature(func=<bound method A.constr of <__main__.A object at 0x7f7685a284d0>>, variables=VarContainer:
  x
  data: None
  metadata: opti0_x_1
  dims: (1, 1)
  is_constant: False


  y
  data: None
  metadata: opti0_p_1
  dims: (1, 1)
  is_constant: True


  , is_objective=False, metadata=MX(((2.*opti0_p_1)-opti0_x_1)))
  

In [6]:
a.opti.value_parameters()

[MX((2==opti0_p_1)), MX((1==opti0_p_2))]